In [1]:
import pandas as pd
from openai import OpenAI

# Analyse der Stichwörter

Dieser Abschnitt befasst sich mit der Generierung von Stichwörtern zu den einzelnen Artikeln. Da NTV keine Stichwörter zu den Inhalten in ihren Artikeln mitliefert, müssen diese zunächst erstellt werden. Anschließend können Zusammenhänge und Themen in einem Graphen dargestellt werden. Zunächst wird der OpenAI-Client erstellt, über den anschließend die Abfragen gesendet werden und die CSV-Datei mit den Artikeln eingelesen. Ebenso wird die Spalte `keywords` hinzugefügt, die zunächst mit `None`-Werten gefüllt wird.

In [2]:
openai_client = OpenAI()

In [3]:
articles_data = pd.read_csv('2-2-data-extended.csv')
articles = pd.DataFrame(articles_data)
articles["keywords"] = None
articles.head()

,date,time,category,headline,text,upload,upload-hour,weekday,length,keywords
0,2024-05-10,17:14:00,politik,Gruppen treten am Vatertag mit Nazi Symbolik auf,Am Vatertag steuern Männer ein altes Militärfa...,2024-05-10 17:14:00,17,Friday,2908,None
1,2024-05-10,17:16:00,politik,Peking schimpft ueber US Zerstoerer im Suedchi...,China beansprucht die Paracel-Inseln im Südchi...,2024-05-10 17:16:00,17,Friday,3504,None
2,2024-05-10,19:33:00,politik,SPD Politiker Ecke schildert Sekunden der Prue...,Der vor einer Woche brutal attackierte SPD-Pol...,2024-05-10 19:33:00,19,Friday,2445,None
3,2024-05-10,08:40:00,politik,Landkreise fordern haertere Buergergeld Sankti...,Beim Bürgergeld in seiner jetzigen Form bemäng...,2024-05-10 08:40:00,8,Friday,2984,None
4,2024-05-10,15:44:00,politik,Buschmanns Regierungsflieger kehrt in Parkposi...,Deutsche Regierungsflieger sind für ihre Panne...,2024-05-10 15:44:00,15,Friday,2570,None


## Generierung der Stichwörter

Die Stichwörter zu den Artikeln sollen mit Hilfe von LLMs generiert werden, in diesem Fall mit Hilfe von ChatGPT. Verwendet wird, vorrangig aus kostengründen, das Modell `gpt-4o-mini`.

In [4]:
def create_keywords_for_articles(openai_client, articles_df, start=0, end=5):
    print()
    for i in range(start, end):
        results = []
        article_text = articles_df.iloc[i]["text"]
        article_headline = articles_df.iloc[i]["headline"]
        completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "Du fasst Nachrichtenartikel des Nachrichtensenders NTV anhand von Stichpunkten zusammenzufassen. Du gibst ausschließlich die Stichwörter durch Kommata getrennt zurück. Die Stichwörter sollen den Inhalt mit übergeordneten Kategorien wie Politik, Wirtschaft, Gesellschaft, Auto, Sport, Unterhaltung und dergleichen beschreiben. Es sollen exakt 10 Stichwörter generiert werden.",
            },
            {
                "role": "user",
                "content": article_text,
                },
            ],
        )
        keywords = completion.choices[0].message.content
        articles_df.at[i, "keywords"] = keywords
        print('\r', end='')
        print(f"Generated Keywords for {i + 1} articles", end='')
    return articles_df

In [27]:
articles = create_keywords_for_articles(openai_client, articles, 8059, 10678)
articles.to_csv('2-3-data-extended-keywords.csv', index=False)


Generated Keywords for 10678 articles

In [25]:
articles.to_csv('2-3-data-extended-keywords.csv', index=False)